In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import json
import csv
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from google.cloud import bigquery
from sqlalchemy import create_engine
from dotenv import load_dotenv
from scipy import stats
%matplotlib inline

In [2]:
load_dotenv(".env2")

host=os.environ.get('DB_HOST')
port=os.environ.get('DB_PORT')
database=os.environ.get('DB_DATABASE')
user=os.environ.get('DB_USER')
password=os.environ.get('DB_PASSWORD')
project_id = os.environ.get('PROJECT_ID')

In [3]:
conn_string = f"postgresql://{user}:{password}@{host}/{database}"
postgres_engine = create_engine(conn_string)

In [88]:
bigquery_id = project_id

# Initialize the BigQuery client
client = bigquery.Client(project=bigquery_id)

# SQL query to get a sample of the data
query = """
select *
from `bigquery-public-data.google_analytics_sample.ga_sessions_*`
where _table_suffix between '20161201' and '20161231'
"""

# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert the results to a pandas DataFrame
df_original = results.to_dataframe()
df_original

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,<NA>,1,1483249525,1483249525,20161231,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Eastern...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 21, 'minu...",452867034752168091,None,None,Organic Search,Not Socially Engaged
1,<NA>,1,1483190275,1483190275,20161231,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Western...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",2720512902919599506,None,None,Direct,Not Socially Engaged
2,<NA>,1,1483194277,1483194277,20161231,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 6, 'minut...",2636626063434222577,None,None,Direct,Not Socially Engaged
3,<NA>,3,1483190773,1483190773,20161231,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",6756705736141944845,None,None,Organic Search,Not Socially Engaged
4,<NA>,1,1483217632,1483217632,20161231,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6619114209511927776,None,None,Organic Search,Not Socially Engaged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79119,<NA>,1,1480954297,1480954297,20161205,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Africa', 'subContinent': 'South...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 8, 'minut...",137461519761333194,None,None,Social,Not Socially Engaged
79120,<NA>,1,1481003630,1481003630,20161205,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Southea...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 21, 'minu...",3595822578469875936,None,None,Social,Not Socially Engaged
79121,<NA>,1,1480929385,1480929385,20161205,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Southea...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 1, 'minut...",808356084423929068,None,None,Social,Not Socially Engaged
79122,<NA>,1,1480942705,1480942705,20161205,"{'visits': 1, 'hits': 4, 'pageviews': 4, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Souther...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 4, 'minut...",5195498074841493010,None,None,Social,Not Socially Engaged


In [89]:
df = df_original.copy()
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,<NA>,1,1483249525,1483249525,20161231,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Eastern...","[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 21, 'minu...",452867034752168091,None,None,Organic Search,Not Socially Engaged
1,<NA>,1,1483190275,1483190275,20161231,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Western...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",2720512902919599506,None,None,Direct,Not Socially Engaged
2,<NA>,1,1483194277,1483194277,20161231,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 6, 'minut...",2636626063434222577,None,None,Direct,Not Socially Engaged
3,<NA>,3,1483190773,1483190773,20161231,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",6756705736141944845,None,None,Organic Search,Not Socially Engaged
4,<NA>,1,1483217632,1483217632,20161231,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6619114209511927776,None,None,Organic Search,Not Socially Engaged


In [ ]:
normalized_data = []

for i in df.index:    
    df_hits = df['hits'][i]
    df_hits = pd.json_normalize(df_hits)
    df_hits.columns = [f"hits_{col}" for col in df_hits.columns]
    df_hits.insert(0, 'visit_id', df['visitId'][i])  # Insert the 'visitId' column as the first column
    normalized_data.append(df_hits)

pd.set_option('display.max_columns', None)

result_df = pd.concat(normalized_data)
result_df

In [106]:
result_df2 = result_df.copy()

normalized_data2 = []

for i in result_df2.index:    
    df_product = result_df2['hits_product'][i]
    df_product = pd.json_normalize(df_product)
    df_product.columns = [f"product_{col}" for col in df_product.columns]
    df_product.insert(0, 'visit_id', df['visitId'][i])  # Insert the 'visitId' column as the first column
    normalized_data2.append(df_product)

df_product = pd.concat(normalized_data2)
df_product

In [96]:
hits_cols = result_df.columns

for cols in hits_cols:
    print(cols)

visit_id
hits_hitNumber
hits_time
hits_hour
hits_minute
hits_isSecure
hits_isInteraction
hits_isEntrance
hits_isExit
hits_referer
hits_transaction
hits_item
hits_contentInfo
hits_eventInfo
hits_product
hits_promotion
hits_promotionActionInfo
hits_refund
hits_experiment
hits_publisher
hits_customVariables
hits_customDimensions
hits_customMetrics
hits_type
hits_latencyTracking
hits_sourcePropertyInfo
hits_dataSource
hits_publisher_infos
hits_page.pagePath
hits_page.hostname
hits_page.pageTitle
hits_page.searchKeyword
hits_page.searchCategory
hits_page.pagePathLevel1
hits_page.pagePathLevel2
hits_page.pagePathLevel3
hits_page.pagePathLevel4
hits_appInfo.name
hits_appInfo.version
hits_appInfo.id
hits_appInfo.installerId
hits_appInfo.appInstallerId
hits_appInfo.appName
hits_appInfo.appVersion
hits_appInfo.appId
hits_appInfo.screenName
hits_appInfo.landingScreenName
hits_appInfo.exitScreenName
hits_appInfo.screenDepth
hits_exceptionInfo.description
hits_exceptionInfo.isFatal
hits_exceptionIn

In [21]:
select_cols = ['visit_id', 'hits_hitNumber', 'hits_type', 'hits_eCommerceAction.action_type', 'hits_time', 'hits_hour', 'hits_minute', 'hits_page.pagePath', 'hits_isEntrance', 
    'hits_isExit', 'hits_appInfo.screenName', 'hits_appInfo.landingScreenName', 'hits_appInfo.exitScreenName']
df_hits = df_hits[select_cols]
df_hits

,visit_id,hits_hitNumber,hits_type,hits_eCommerceAction.action_type,hits_time,hits_hour,hits_minute,hits_page.pagePath,hits_isEntrance,hits_isExit,hits_appInfo.screenName,hits_appInfo.landingScreenName,hits_appInfo.exitScreenName
0,1483087697,1,PAGE,0,0,0,48,/home,True,None,www.googlemerchandisestore.com/home,www.googlemerchandisestore.com/home,www.googlemerchandisestore.com/home
1,1483087697,2,PAGE,0,7534,0,48,/home,None,None,shop.googlemerchandisestore.com/home,www.googlemerchandisestore.com/home,www.googlemerchandisestore.com/home
2,1483087697,3,PAGE,0,14983,0,48,/google+redesign/accessories/stickers,None,None,shop.googlemerchandisestore.com/google+redesig...,www.googlemerchandisestore.com/home,www.googlemerchandisestore.com/home
3,1483087697,4,PAGE,0,38484,0,48,/google+redesign/bags,None,None,shop.googlemerchandisestore.com/google+redesig...,www.googlemerchandisestore.com/home,www.googlemerchandisestore.com/home
4,1483087697,5,PAGE,0,52012,0,49,/google+redesign/electronics,None,None,shop.googlemerchandisestore.com/google+redesig...,www.googlemerchandisestore.com/home,www.googlemerchandisestore.com/home
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,1481598847,83,PAGE,0,1914226,19,46,/basket.html,None,None,shop.googlemerchandisestore.com/basket.html,www.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/basket.html
83,1481598847,84,PAGE,0,1948858,19,46,/basket.html,None,None,shop.googlemerchandisestore.com/basket.html,www.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/basket.html
84,1481598847,85,EVENT,4,2039666,19,48,/basket.html,None,None,shop.googlemerchandisestore.com/basket.html,www.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/basket.html
85,1481598847,86,PAGE,0,2046742,19,48,/basket.html,None,None,shop.googlemerchandisestore.com/basket.html,www.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/basket.html


In [22]:
df_hits.to_sql("hits", postgres_engine, if_exists='replace', index=False)
print("Completed exporting to sql server")

Completed exporting to sql server


In [91]:
json_cols = ['totals', 'trafficSource', 'device', 'geoNetwork']

for column in json_cols:
    normalized_df = pd.json_normalize(df[column])
    normalized_df.columns = [f"{column}_{col}" for col in normalized_df.columns]
    df = pd.concat([df.drop(columns=column), normalized_df], axis=1)

df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType,totals_visits,totals_hits,totals_pageviews,totals_timeOnSite,totals_bounces,totals_transactions,totals_transactionRevenue,totals_newVisits,totals_screenviews,totals_uniqueScreenviews,totals_timeOnScreen,totals_totalTransactionRevenue,totals_sessionQualityDim,trafficSource_referralPath,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adContent,trafficSource_isTrueDirect,trafficSource_campaignCode,trafficSource_adwordsClickInfo.campaignId,trafficSource_adwordsClickInfo.adGroupId,trafficSource_adwordsClickInfo.creativeId,trafficSource_adwordsClickInfo.criteriaId,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.customerId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.targetingCriteria,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.targetingCriteria.boomUserlistId,device_browser,device_browserVersion,device_browserSize,device_operatingSystem,device_operatingSystemVersion,device_isMobile,device_mobileDeviceBranding,device_mobileDeviceModel,device_mobileInputSelector,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_flashVersion,device_javaEnabled,device_language,device_screenColors,device_screenResolution,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_cityId,geoNetwork_networkDomain,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_networkLocation
0,<NA>,1,1483249525,1483249525,20161231,"[{'index': 4, 'value': 'APAC'}]","[{'hitNumber': 1, 'time': 0, 'hour': 21, 'minu...",452867034752168091,None,None,Organic Search,Not Socially Engaged,1,1,1.0,NaN,1.0,NaN,NaN,1.0,None,None,None,NaN,None,None,(not set),yahoo,organic,(not provided),None,None,None,None,None,None,None,NaN,None,not available in demo dataset,None,None,None,NaN,None,NaN,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Asia,Eastern Asia,Japan,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,eonet.ne.jp,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,<NA>,1,1483190275,1483190275,20161231,[],"[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",2720512902919599506,None,None,Direct,Not Socially Engaged,1,1,1.0,NaN,1.0,NaN,NaN,1.0,None,None,None,NaN,None,None,(not set),(direct),(none),None,None,True,None,None,None,None,None,NaN,None,not available in demo dataset,None,None,None,NaN,None,NaN,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Asia,Western Asia,Israel,Tel Aviv District,(not set),Tel Aviv-Yafo,not available in demo dataset,bezeqint.net,not available in demo dataset,not available in demo dataset,not available in demo dataset
2,<NA>,1,1483194277,1483194277,20161231,"[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 6, 'minut...",2636626063434222577,None,None,Direct,Not Socially Engaged,1,1,1.0,NaN,1.0,NaN,NaN,1.0,None,None,None,NaN,None,None,(not set),(direct

In [30]:
pd.set_option('display.max_columns', None)  # None means unlimited columns

In [31]:
drop_cols = ['visitorId', 'customDimensions', 'hits']
df = df.drop(columns = drop_cols)
df.head()

,visitNumber,visitId,visitStartTime,date,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType,totals_visits,totals_hits,totals_pageviews,totals_timeOnSite,totals_bounces,totals_transactions,totals_transactionRevenue,totals_newVisits,totals_screenviews,totals_uniqueScreenviews,totals_timeOnScreen,totals_totalTransactionRevenue,totals_sessionQualityDim,trafficSource_referralPath,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adContent,trafficSource_isTrueDirect,trafficSource_campaignCode,trafficSource_adwordsClickInfo.campaignId,trafficSource_adwordsClickInfo.adGroupId,trafficSource_adwordsClickInfo.creativeId,trafficSource_adwordsClickInfo.criteriaId,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.customerId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.targetingCriteria,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.targetingCriteria.boomUserlistId,device_browser,device_browserVersion,device_browserSize,device_operatingSystem,device_operatingSystemVersion,device_isMobile,device_mobileDeviceBranding,device_mobileDeviceModel,device_mobileInputSelector,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_flashVersion,device_javaEnabled,device_language,device_screenColors,device_screenResolution,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_cityId,geoNetwork_networkDomain,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_networkLocation
0,2,1483087697,1483087697,20161230,7385871189843591760,None,None,Organic Search,Not Socially Engaged,1,14,13.0,922.0,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,(not set),(direct),(none),None,None,None,None,None,None,None,None,NaN,None,not available in demo dataset,None,None,None,NaN,None,NaN,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,San Francisco,not available in demo dataset,comcastbusiness.net,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,1,1483086672,1483086672,20161230,7925884217149710796,None,None,Organic Search,Not Socially Engaged,1,14,11.0,301.0,NaN,NaN,NaN,1.0,None,None,None,NaN,None,None,(not set),(direct),(none),None,None,None,None,None,None,None,None,NaN,None,not available in demo dataset,None,None,None,NaN,None,NaN,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,None,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Europe,Southern Europe,Macedonia (FYROM),not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
2,2,1483128188,1483128188,20161230,224580975515679767,None,None,Organic Search,Not Socially Engaged,1,14,9.0,412.0,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,(not set),(direct),(none),None,None,None,None,None,None,None,None,NaN,None,not available in demo dataset,None,None,None,NaN,None,NaN,Safari,not available in demo dataset,not available in demo dataset,iOS,not available i

In [34]:
df_cols = df.columns

for cols in df_cols:
    print(cols)

visitNumber
visitId
visitStartTime
date
fullVisitorId
userId
clientId
channelGrouping
socialEngagementType
totals_visits
totals_hits
totals_pageviews
totals_timeOnSite
totals_bounces
totals_transactions
totals_transactionRevenue
totals_newVisits
totals_screenviews
totals_uniqueScreenviews
totals_timeOnScreen
totals_totalTransactionRevenue
totals_sessionQualityDim
trafficSource_referralPath
trafficSource_campaign
trafficSource_source
trafficSource_medium
trafficSource_keyword
trafficSource_adContent
trafficSource_isTrueDirect
trafficSource_campaignCode
trafficSource_adwordsClickInfo.campaignId
trafficSource_adwordsClickInfo.adGroupId
trafficSource_adwordsClickInfo.creativeId
trafficSource_adwordsClickInfo.criteriaId
trafficSource_adwordsClickInfo.page
trafficSource_adwordsClickInfo.slot
trafficSource_adwordsClickInfo.criteriaParameters
trafficSource_adwordsClickInfo.gclId
trafficSource_adwordsClickInfo.customerId
trafficSource_adwordsClickInfo.adNetworkType
trafficSource_adwordsClickInf

In [56]:
df['totals_uniqueScreenviews'].unique()

array([None], dtype=object)

In [37]:
df_sess = df.copy()
new_cols = {'visitId': 'visit_id', 'fullVisitorId': 'full_visit_id', 'visitNumber': 'visit_number', 'visitStartTime': 'visit_start_time', 'date':'visit_date',
            'channelGrouping': 'channel_group'}
df_sess.rename(columns=new_cols, inplace=True)
select_cols = ['visit_id', 'full_visit_id', 'visit_number', 'visit_start_time', 'visit_date', 'trafficSource_source', 'trafficSource_medium', 'trafficSource_campaign', 
    'trafficSource_adContent', 'trafficSource_isTrueDirect', 'totals_bounces', 'totals_hits', 'totals_newVisits', 'totals_transactionRevenue',
    'totals_transactions', 'device_browser', 'device_deviceCategory', 'device_operatingSystem', 'channel_group']
df_sess = df_sess[select_cols]
df_sess

,visit_id,full_visit_id,visit_number,visit_start_time,visit_date,trafficSource_source,trafficSource_medium,trafficSource_campaign,trafficSource_adContent,trafficSource_isTrueDirect,totals_bounces,totals_hits,totals_newVisits,totals_transactionRevenue,totals_transactions,device_browser,device_deviceCategory,device_operatingSystem,channel_group
0,1483087697,7385871189843591760,2,1483087697,20161230,(direct),(none),(not set),None,None,NaN,14,NaN,NaN,NaN,Chrome,desktop,Macintosh,Organic Search
1,1483086672,7925884217149710796,1,1483086672,20161230,(direct),(none),(not set),None,None,NaN,14,1.0,NaN,NaN,Chrome,desktop,Windows,Organic Search
2,1483128188,224580975515679767,2,1483128188,20161230,(direct),(none),(not set),None,None,NaN,14,NaN,NaN,NaN,Safari,mobile,iOS,Organic Search
3,1483126904,1712564087609515628,1,1483126904,20161230,(direct),(none),(not set),None,None,NaN,14,1.0,NaN,NaN,Chrome,mobile,Android,Organic Search
4,1483145960,7139371055947807427,1,1483145960,20161230,(direct),(none),(not set),None,None,NaN,17,1.0,NaN,NaN,Chrome,tablet,Android,Paid Search
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79119,1481534615,6439926865693529882,1,1481534615,20161212,(direct),(none),(not set),None,None,NaN,45,1.0,NaN,NaN,Safari,mobile,iOS,Paid Search
79120,1481532768,4601438455383424117,3,1481532768,20161212,(direct),(none),(not set),None,True,NaN,45,NaN,NaN,NaN,Internet Explorer,mobile,Windows,Organic Search
79121,1481566509,2968685844689158816,1,1481566509,20161212,(direct),(none),(not set),None,None,NaN,56,1.0,NaN,NaN,Chrome,mobile,Android,Organic Search
79122,1481540012,0071895336593296958,1,1481540012,20161212,youtube.com,referral,(not set),None,None,NaN,74,1.0,NaN,NaN,Chrome,mobile,Android,Social


In [54]:
from datetime import datetime
df_sess['visit_start_time'] = pd.to_datetime(df_sess['visit_start_time'], unit='s')

print(df_sess.dtypes)

visit_id                              Int64
full_visit_id                        object
visit_number                          Int64
visit_start_time              datetime64[s]
visit_date                           object
trafficSource_source                 object
trafficSource_medium                 object
trafficSource_campaign               object
trafficSource_adContent              object
trafficSource_isTrueDirect           object
totals_bounces                      float64
totals_hits                           int64
totals_newVisits                    float64
totals_transactionRevenue           float64
totals_transactions                 float64
device_browser                       object
device_deviceCategory                object
device_operatingSystem               object
channel_group                        object
dtype: object


In [57]:
df_sess.to_sql("session", postgres_engine, if_exists='replace', index=False)
print("Completed exporting to sql server")

Completed exporting to sql server


In [95]:
df_geo_network = df.copy()

df_geo_network.rename(columns={'visitId': 'visit_id'}, inplace=True)

select_cols = ['visit_id', 'geoNetwork_continent', 'geoNetwork_country']
df_geo_network = df_geo_network[select_cols]
df_geo_network.head()

,visit_id,geoNetwork_continent,geoNetwork_country
0,1483249525,Asia,Japan
1,1483190275,Asia,Israel
2,1483194277,Americas,United States
3,1483190773,Europe,Guernsey
4,1483217632,Europe,United Kingdom


In [96]:
df_geo_network.to_sql("geo", postgres_engine, if_exists='replace', index=False)
print("Completed exporting to sql server")

Completed exporting to sql server


In [58]:
query = """
select *
from "session"
"""

df = pd.read_sql_query(sql=query, con=postgres_engine)
df

,visit_id,full_visit_id,visit_number,visit_start_time,visit_date,trafficSource_source,trafficSource_medium,trafficSource_campaign,trafficSource_adContent,trafficSource_isTrueDirect,totals_bounces,totals_hits,totals_newVisits,totals_transactionRevenue,totals_transactions,device_browser,device_deviceCategory,device_operatingSystem,channel_group
0,1483087697,7385871189843591760,2,2016-12-30 08:48:17,20161230,(direct),(none),(not set),None,None,NaN,14,NaN,NaN,NaN,Chrome,desktop,Macintosh,Organic Search
1,1483086672,7925884217149710796,1,2016-12-30 08:31:12,20161230,(direct),(none),(not set),None,None,NaN,14,1.0,NaN,NaN,Chrome,desktop,Windows,Organic Search
2,1483128188,224580975515679767,2,2016-12-30 20:03:08,20161230,(direct),(none),(not set),None,None,NaN,14,NaN,NaN,NaN,Safari,mobile,iOS,Organic Search
3,1483126904,1712564087609515628,1,2016-12-30 19:41:44,20161230,(direct),(none),(not set),None,None,NaN,14,1.0,NaN,NaN,Chrome,mobile,Android,Organic Search
4,1483145960,7139371055947807427,1,2016-12-31 00:59:20,20161230,(direct),(none),(not set),None,None,NaN,17,1.0,NaN,NaN,Chrome,tablet,Android,Paid Search
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79119,1481534615,6439926865693529882,1,2016-12-12 09:23:35,20161212,(direct),(none),(not set),None,None,NaN,45,1.0,NaN,NaN,Safari,mobile,iOS,Paid Search
79120,1481532768,4601438455383424117,3,2016-12-12 08:52:48,20161212,(direct),(none),(not set),None,True,NaN,45,NaN,NaN,NaN,Internet Explorer,mobile,Windows,Organic Search
79121,1481566509,2968685844689158816,1,2016-12-12 18:15:09,20161212,(direct),(none),(not set),None,None,NaN,56,1.0,NaN,NaN,Chrome,mobile,Android,Organic Search
79122,1481540012,0071895336593296958,1,2016-12-12 10:53:32,20161212,youtube.com,referral,(not set),None,None,NaN,74,1.0,NaN,NaN,Chrome,mobile,Android,Social


In [86]:
# Daily user count, daily visit number, average daily visit number

query = """
with 
temp_01 as (
	select to_char(date_trunc('day', visit_start_time), 'yyyy-mm-dd') as d_day
    	, count(distinct visit_id) as daily_user_cnt
    	, count(visit_number) as daily_visit_number 
	from session group by to_char(date_trunc('day', visit_start_time), 'yyyy-mm-dd')
)
select * 
	, 1.0*daily_visit_number/daily_user_cnt as avg_daily_visit_number
from temp_01
"""

df = pd.read_sql_query(sql=query, con=postgres_engine)
df.head(10)

,d_day,daily_user_cnt,daily_visit_number,avg_daily_visit_number
0,2016-12-01,2968,3045,1.025943
1,2016-12-02,3782,3866,1.022210
2,2016-12-03,3152,3210,1.018401
3,2016-12-04,2919,2980,1.020898
4,2016-12-05,3994,4096,1.025538
5,2016-12-06,3444,3517,1.021196
6,2016-12-07,2730,2786,1.020513
7,2016-12-08,2970,3043,1.024579
8,2016-12-09,2926,2983,1.019481
9,2016-12-10,2224,2266,1.018885


In [88]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(
    x=df['d_day'],
    y=df['daily_user_cnt'],
    name='daily user count'), secondary_y=False)
fig.add_trace(go.Scatter(
    x=df['d_day'],
    y=df['daily_visit_number'],
    name='daily visit number'), secondary_y=False)
fig.add_trace(go.Scatter(
    x=df['d_day'],
    y=df['avg_daily_visit_number'],
    name='avg daily visit number'), secondary_y=True)



fig.update_yaxes(range=(1.0, 1.2),  secondary_y=True)
fig.update_xaxes(type='category')

fig.show()

In [79]:
# DAU (Daily Active Users)

query = """
select date_trunc('day', visit_start_time)::date as d_day, count(distinct visit_id) as dail_user_cnt
from session 
group by date_trunc('day', visit_start_time)::date
"""

df = pd.read_sql_query(sql=query, con=postgres_engine)
df

,d_day,dail_user_cnt
0,2016-12-01,2968
1,2016-12-02,3782
2,2016-12-03,3152
3,2016-12-04,2919
4,2016-12-05,3994
5,2016-12-06,3444
6,2016-12-07,2730
7,2016-12-08,2970
8,2016-12-09,2926
9,2016-12-10,2224


In [90]:
# WAU (Weekly Active Users)

query = """
select date_trunc('week', visit_start_time)::date as week_day, count(distinct visit_id) as user_cnt 
from session 
group by date_trunc('week', visit_start_time)::date;
"""

df = pd.read_sql_query(sql=query, con=postgres_engine)
df

,week_day,user_cnt
0,2016-11-28,12821
1,2016-12-05,20373
2,2016-12-12,19516
3,2016-12-19,15252
4,2016-12-26,9627


In [81]:
# MAU (Monthly Active Users)

query = """
select date_trunc('month', visit_start_time)::date as month_day, count(distinct visit_id) as user_cnt 
from session 
group by date_trunc('month', visit_start_time)::date;
"""

df = pd.read_sql_query(sql=query, con=postgres_engine)
df

,month_day,user_cnt
0,2016-12-01,77283
1,2017-01-01,306


In [62]:
# Generate DAU by past date, WAU by past 7 days, MAU by past 30 days on a daily basis
# Stickiness Metric (How many users out of the monthly users visit periodically?)

query = """
create table daily_dau as

with temp_00 as (
select generate_series('2016-12-01'::date , '2016-12-31'::date, '1 day'::interval)::date as curr_date
)
select b.curr_date, count(distinct visit_id) as dau
from session a
	cross join temp_00 b
where visit_start_time >= (b.curr_date - interval '1 days') and visit_start_time < b.curr_date
group by b.curr_date
;

create table daily_wau as
with temp_00 as (
select generate_series('2016-12-01'::date , '2016-12-31'::date, '1 day'::interval)::date as curr_date
)
select b.curr_date, count(distinct visit_id) as wau
from session a
	cross join temp_00 b
where visit_start_time >= (b.curr_date - interval '7 days') and visit_start_time < b.curr_date
group by b.curr_date
;

create table daily_mau as
with temp_00 as (
select generate_series('2016-12-01'::date , '2016-12-31'::date, '1 day'::interval)::date as curr_date
)
select b.curr_date, count(distinct visit_id) as mau
from session a
	cross join temp_00 b
where visit_start_time >= (b.curr_date - interval '30 days') and visit_start_time < b.curr_date
group by b.curr_date
;

create table daily_acquisitions as
select a.curr_date, a.dau, b.wau, c.mau
from daily_dau a
	join daily_wau b on a.curr_date = b.curr_date
	join daily_mau c on a.curr_date = c.curr_date
;

select *, round(100.0 * dau/mau, 2) as stickieness
	, round(avg(100.0 * dau/mau) over(), 2) as avg_stickieness
from daily_acquisitions
where curr_date between to_date('2016-12-01', 'yyyy-mm-dd') and to_date('2016-12-31', 'yyyy-mm-dd')
"""

df = pd.read_sql_query(sql=query, con=postgres_engine)
df

,curr_date,dau,wau,mau,stickieness,avg_stickieness
0,2016-12-02,2968,2968,2968,100.00,12.63
1,2016-12-03,3782,6750,6750,56.03,12.63
2,2016-12-04,3152,9902,9902,31.83,12.63
3,2016-12-05,2919,12821,12821,22.77,12.63
4,2016-12-06,3994,16815,16815,23.75,12.63
5,2016-12-07,3444,20259,20259,17.00,12.63
6,2016-12-08,2730,22989,22989,11.88,12.63
7,2016-12-09,2970,22991,25959,11.44,12.63
8,2016-12-10,2926,22135,28885,10.13,12.63
9,2016-12-11,2224,21207,31109,7.15,12.63


In [63]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()

fig.add_trace(go.Scatter(x=df['curr_date'], y=df['stickieness'], name='daily stickieness'))
fig.add_trace(go.Scatter(x=df['curr_date'], y=df['avg_stickieness'],  name='avg stickieness'))
fig.show()

In [11]:
query = """
with temp_01 as (
	select visit_id, date_trunc('week', visit_start_time)::date as week, count(*) as weekly_user_cnt  
	from session  		
	group by visit_id, date_trunc('week', visit_start_time)::date 
), 
temp_02 as ( 
	select week
		, case when weekly_user_cnt = 1 then '0_only_first_session'
		    	when weekly_user_cnt between 2 and 3 then '2_between_3'
		    	when weekly_user_cnt between 4 and 8 then '4_between_8'
		    	when weekly_user_cnt between 9 and 14 then '9_between_14'
		    	when weekly_user_cnt between 15 and 25 then '15_between_25'
		    	when weekly_user_cnt >= 26 then 'over_26' end as range
		, count(*) as user_cnt 
	from temp_01 
	group by week
			 , case when weekly_user_cnt = 1 then '0_only_first_session'
					when weekly_user_cnt between 2 and 3 then '2_between_3'
					when weekly_user_cnt between 4 and 8 then '4_between_8'
					when weekly_user_cnt between 9 and 14 then '9_between_14'
			    	when weekly_user_cnt between 15 and 25 then '15_between_25'
			    	when weekly_user_cnt >= 26 then 'over_26' end
)
select week, 
	sum(case when range='0_only_first_session' then user_cnt else 0 end) as "0_only_first_session"
	,sum(case when range='2_between_3' then user_cnt else 0 end) as "2_between_3"
	,sum(case when range='4_between_8' then user_cnt else 0 end) as "4_between_8"
	,sum(case when range='9_between_14' then user_cnt else 0 end) as "9_between_14"
	,sum(case when range='15_between_25' then user_cnt else 0 end) as "15_between_25"
	,sum(case when range='over_26' then user_cnt else 0 end) as "over_26"
from temp_02 
group by week order by 1
"""
df = pd.read_sql_query(sql=query, con=postgres_engine)
df.head(10)

,week,0_only_first_session,2_between_3,4_between_8,9_between_14,15_between_25,over_26
0,2016-11-28,12547.0,274.0,0.0,0.0,0.0,0.0
1,2016-12-05,19944.0,427.0,2.0,0.0,0.0,0.0
2,2016-12-12,19090.0,425.0,1.0,0.0,0.0,0.0
3,2016-12-19,14994.0,258.0,0.0,0.0,0.0,0.0
4,2016-12-26,9513.0,114.0,0.0,0.0,0.0,0.0


In [14]:
import plotly.express as px

color_scale = px.colors.qualitative.Set3

fig = px.bar(df, x="week", y=["0_only_first_session", "2_between_3", "4_between_8", "9_between_14", "15_between_25", "over_26"],
             title="weekly user session count distribution",
             color_discrete_sequence=color_scale,
             labels={"value": "User Count"},
             text="value",  # Show data labels
             barmode="group")

fig.update_xaxes(tickangle=-45)

fig.show()

In [28]:
# Extract the average, maximum, minimum, and quartile percentile times it takes 
# for a user to connect to their second session after their first session.

query = """
with
temp_01 as (
select visit_id, row_number() over (partition by visit_id order by visit_start_time) as session_rnum 
	, visit_start_time	
	, count(*) over (partition by visit_id) as session_cnt
from session 
),
temp_02 as (
select visit_id
    , max(visit_start_time) - min(visit_start_time) as sess_time_diff
from temp_01 where session_rnum <=2 and session_cnt > 1
group by visit_id
)
select justify_interval(avg(sess_time_diff)) as avg_time
    , max(sess_time_diff) as max_time, min(sess_time_diff) as min_time 
	, percentile_disc(0.25) within group (order by sess_time_diff) as percentile_1
	, percentile_disc(0.5) within group (order by sess_time_diff)	as percentile_2
	, percentile_disc(0.75) within group (order by sess_time_diff)	as percentile_3
	, percentile_disc(1.0) within group (order by sess_time_diff)	as percentile_4
from temp_02
where sess_time_diff::interval > interval '0 second';
"""
df = pd.read_sql_query(sql=query, con=postgres_engine)
df.head()

,avg_time,max_time,min_time,percentile_1,percentile_2,percentile_3,percentile_4
0,0 days 00:13:55.531646,0 days 01:18:42,0 days 00:00:01,0 days 00:01:52,0 days 00:08:07,0 days 00:18:27,0 days 01:18:42


In [94]:
query = """
with 
temp_01 as ( 
select b.channel_group, date_trunc('week', b.visit_start_time)::date as conversion_week
	, count(distinct a.visit_id) as visit_cnt
	, count(distinct case when a.hits_action_type='6' then a.visit_id end) as purchase_user_cnt
from hits a
	join session b on a.visit_id = b.visit_id
group by b.channel_group, date_trunc('week', b.visit_start_time)::date
)
select channel_group, conversion_week, purchase_user_cnt, visit_cnt
	, round(100.0* purchase_user_cnt/visit_cnt, 2) as sale_conversion_rate	
from temp_01 a	
order by 1, 2
;
"""

df_channel = pd.read_sql_query(sql=query, con=postgres_engine)
df_channel

,channel_group,conversion_week,purchase_user_cnt,visit_cnt,sale_conversion_rate
0,Affiliates,2016-11-28,0,112,0.00
1,Affiliates,2016-12-05,0,271,0.00
2,Affiliates,2016-12-12,0,239,0.00
3,Affiliates,2016-12-19,0,182,0.00
4,Affiliates,2016-12-26,0,134,0.00
5,Direct,2016-11-28,17,1365,1.25
6,Direct,2016-12-05,69,3220,2.14
7,Direct,2016-12-12,81,4393,1.84
8,Direct,2016-12-19,39,3461,1.13
9,Direct,2016-12-26,18,1690,1.07


In [100]:
import plotly.express as px

fig = px.line(data_frame=df_channel, x='conversion_week', y='sale_conversion_rate', color='channel_group', markers=True)
fig.show()

In [69]:
# total user count by devices and ratio

query = """
with temp_01 as (
select count(*) as total_cnt 
from session
),
temp_02 as (
select device_category, count(*) as device_cnt
from session 
group by device_category 
)
select device_category, device_cnt, 1.0*device_cnt/total_cnt as device_ratio_by_total
from temp_01, temp_02;
"""

df_device = pd.read_sql_query(sql=query, con=postgres_engine)
df_device.head()

,device_category,device_cnt,device_ratio_by_total
0,mobile,19594,0.247637
1,desktop,56440,0.713311
2,tablet,3090,0.039053


In [72]:
# weekly user count by devices

query = """
select date_trunc('week', visit_start_time) as date
	, sum(case when device_category = 'desktop' then 1 else 0 end) as desktop_cnt
	, sum(case when device_category = 'mobile' then 1 else 0 end) as mobile_cnt
	, sum(case when device_category = 'tablet' then 1 else 0 end) as tablet_cnt
	, count(*)
from session 
group by date_trunc('week', visit_start_time) order by 1;
"""

df_device = pd.read_sql_query(sql=query, con=postgres_engine)
df_device.head()

,date,desktop_cnt,mobile_cnt,tablet_cnt,count
0,2016-11-28,10567,2141,393,13101
1,2016-12-05,15622,4425,765,20812
2,2016-12-12,13736,5423,799,19958
3,2016-12-19,10386,4516,610,15512
4,2016-12-26,6129,3089,523,9741


In [85]:
import plotly.express as px

color_scale = px.colors.qualitative.Set3

fig = px.bar(df_device, x="date", y=["desktop_cnt", "mobile_cnt", "tablet_cnt"],
             title="Weekly user count by devices",
             color_discrete_sequence=color_scale,
             labels={"value": "User Count"},
             text="value",  # Show data labels
             barmode="group")

fig.update_xaxes(tickangle=-45)

fig.show()

In [136]:
query = """
select geo_country, count(distinct b.visit_id)
from session a 
left join geo b on a.visit_id = b.visit_id
group by geo_country order by count(distinct b.visit_id) desc;
"""

df_geo = pd.read_sql_query(sql=query, con=postgres_engine)
df_geo.head()

,geo_country,count
0,United States,37333
1,India,3610
2,United Kingdom,3005
3,Canada,2495
4,Vietnam,2178


In [139]:
fig = px.choropleth(data_frame=df_geo, locations='geo_country', locationmode='country names', 
                    color='count', color_discrete_map='Viridis',
                    title='geo location by user count')

fig.show()

In [142]:
# top 5 pages that has the most hits 

query = """
select hits_page_path, count(*) as hits_by_page 
from hits
group by hits_page_path order by 2 desc
FETCH FIRST 5 ROW only;
"""

df_hit = pd.read_sql_query(sql=query, con=postgres_engine)
df_hit.head()

,hits_page_path,hits_by_page
0,/home,94959
1,/basket.html,26639
2,/google+redesign/shop+by+brand/youtube,16206
3,/google+redesign/apparel/men++s/men++s+t+shirts,12967
4,/signin.html,11695


In [196]:
# Page hit counts per day for the past 30 days and the average daily page hit count over 30 days

query = """
select date_trunc('day', b.visit_start_time)::date as d_day, count(*) as page_cnt
	, round(avg(count(*)) over (), 2) as avg_page_cnt
from hits a
	join session b on a.visit_id = b.visit_id
where b.visit_start_time >= (to_date('20161231', 'yyyymmdd') - interval '30 days') and b.visit_start_time < to_date('20161231', 'yyyymmdd')
and a.hits_type = 'PAGE'
group by date_trunc('day', b.visit_start_time)::date order by 1;
"""

df_hit2 = pd.read_sql_query(sql=query, con=postgres_engine)
df_hit2.head()


,d_day,page_cnt,avg_page_cnt
0,2016-12-01,11181,11400.2
1,2016-12-02,14587,11400.2
2,2016-12-03,10891,11400.2
3,2016-12-04,10398,11400.2
4,2016-12-05,16554,11400.2


In [198]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_hit2['d_day'], y=df_hit2['page_cnt'], name='dialy page counts'))
fig.add_trace(go.Scatter(x=df_hit2['d_day'], y=df_hit2['avg_page_cnt'],  name='avg page counts'))
fig.show()

In [4]:
# page view count and unique page view count per page for the past 30 days

query = """
with 
temp_01 as (
    select a.hits_page_path, COUNT(*) as page_cnt
    from hits a
    join session b on a.visit_id = b.visit_id 
    where visit_start_time >= (to_date('20161231', 'yyyymmdd') - interval '30 days') and visit_start_time < to_date('20161231', 'yyyymmdd')
    and hits_type = 'PAGE'
    group by hits_page_path
), 
temp_02 as (
    select hits_page_path, COUNT(*) as unique_page_cnt
    from (
        select distinct a.visit_id, a.hits_page_path
        from hits a
        join session b on a.visit_id = b.visit_id 
        where visit_start_time >= (to_date('20161231', 'yyyymmdd') - interval '30 days') and visit_start_time < to_date('20161231', 'yyyymmdd')
        and hits_type = 'PAGE'
    ) as subquery
    group by hits_page_path     
)
select a.hits_page_path, page_cnt, unique_page_cnt
from temp_01 a
join temp_02 b ON a.hits_page_path = b.hits_page_path
order by page_cnt DESC;
"""

df_hit = pd.read_sql_query(sql=query, con=postgres_engine)
df_hit.head(10)

,hits_page_path,page_cnt,unique_page_cnt
0,/home,90518,53488
1,/basket.html,25323,7699
2,/google+redesign/shop+by+brand/youtube,13631,10056
3,/signin.html,11820,8619
4,/google+redesign/apparel/men++s/men++s+t+shirts,8946,6378
5,/store.html,7971,4785
6,/google+redesign/apparel/men++s/men++s+outerwear,6787,5035
7,/asearch.html,6549,4222
8,/google+redesign/apparel,5659,3921
9,/google+redesign/electronics,4472,3474


In [33]:
# average time spent per page over the past 30 days

query = """
with
temp_01 as (
	select a.hits_page_path, count(*) as page_cnt
	from hits a
		join session b on a.visit_id = b.visit_id 
	where visit_start_time >= (to_date('20170101', 'yyyymmdd') - interval '30 days') and visit_start_time < to_date('20170101', 'yyyymmdd')
	and a.hits_type = 'PAGE'
	group by hits_page_path
), 
temp_02 as (
	select hits_page_path, count(*) as unique_page_cnt
	from (
		select distinct a.visit_id, a.hits_page_path
		from hits a
			join session b on a.visit_id = b.visit_id 
		where visit_start_time >= (to_date('20170101', 'yyyymmdd') - interval '30 days') and visit_start_time < to_date('20170101', 'yyyymmdd')
		and a.hits_type = 'PAGE'
	) as subquery
	group by hits_page_path
), 
temp_03 as (
	select a.visit_id, hits_page_path, hits_seq, hits_time
		, lead(hits_time) over (partition by a.visit_id order by hits_seq) as next_hit_time
	from hits a
		join session b on a.visit_id = b.visit_id 
	where visit_start_time >= (to_date('20170101', 'yyyymmdd') - interval '30 days') and visit_start_time < to_date('20170101', 'yyyymmdd')
	and a.hits_type = 'PAGE'
), 
temp_04 as (
select hits_page_path, count(*) as page_cnt
	, round(avg(next_hit_time - hits_time)/1000.0, 2) as avg_elapsed_sec
from temp_03
group by hits_page_path
)
select a.hits_page_path, a.page_cnt, b.unique_page_cnt, c.avg_elapsed_sec
from temp_01 a
	left join temp_02 b on a.hits_page_path = b.hits_page_path
	left join temp_04 c on a.hits_page_path = c.hits_page_path
order by 2 desc;
"""

df = pd.read_sql_query(sql=query, con=postgres_engine)
df2 = df.head(5)
df2

,hits_page_path,page_cnt,unique_page_cnt,avg_elapsed_sec
0,/home,87960,51867,36.97
1,/basket.html,24859,7588,34.70
2,/google+redesign/shop+by+brand/youtube,13357,9838,55.61
3,/signin.html,11679,8503,36.54
4,/google+redesign/apparel/men++s/men++s+t+shirts,8759,6261,53.74


In [35]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='hits_page_count', x=df2['hits_page_path'], y=df2['page_cnt']), secondary_y=False)
fig.add_trace(go.Scatter(name='avg_elapsed_time', x=df2['hits_page_path'], y=df2['avg_elapsed_sec']), secondary_y=True)

In [78]:
# past 7 days conversion rate

query = """
with 
temp_01 as ( 
select date_trunc('day', b.visit_start_time)::date as conversion_day
	, count(distinct a.visit_id) as user_cnt
	, count(distinct case when a.hits_action_type = '6' then a.visit_id end) as purchase_cnt
	, sum(totals_revenue) as sum_revenue
from hits a
	join session b on a.visit_id = b.visit_id
where b.visit_start_time >= (to_date('20170101', 'yyyymmdd') - interval '7 days') and b.visit_start_time < to_date('20170101', 'yyyymmdd')
group by date_trunc('day', b.visit_start_time)::date
)
select conversion_day, user_cnt, purchase_cnt, 100.0* purchase_cnt/user_cnt as sale_conversion_rate
from temp_01;
"""

df_conversion = pd.read_sql_query(sql=query, con=postgres_engine)
df_conversion

,conversion_day,user_cnt,purchase_cnt,sale_conversion_rate
0,2016-12-25,1279,19,1.485536
1,2016-12-26,1526,13,0.851900
2,2016-12-27,1808,22,1.216814
3,2016-12-28,1783,23,1.289961
4,2016-12-29,1663,19,1.142514
5,2016-12-30,1197,14,1.169591
6,2016-12-31,1344,14,1.041667


In [80]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(name='daily_user_count', x=df_conversion['conversion_day'], y=df_conversion['user_cnt']), secondary_y=False)
fig.add_trace(go.Scatter(name='daily_conversion_rate', x=df_conversion['conversion_day'], y=df_conversion['sale_conversion_rate']), secondary_y=True)

In [77]:
# overall bounce rate

query = """
with 
temp_01 as ( 
select visit_id, count(*) as page_cnt
from hits
group by visit_id
)
select sum(case when page_cnt = 1 then 1 else 0 end) as bounce_visit_cnt 
	, count(*) as visit_cnt 
	, round(100.0*sum(case when page_cnt = 1 then 1 else 0 end)/count(*), 2) as bounce_sess_pct -- 이탈율
from temp_01;
"""

df_bounce = pd.read_sql_query(sql=query, con=postgres_engine)
df_bounce

,bounce_visit_cnt,visit_cnt,bounce_sess_pct
0,33958,77589,43.77


In [111]:
# past 30 days bounce rate per page 

query = """
with 
temp_01 as ( 
select a.hits_page_path, b.visit_id, hits_seq, hits_type, hits_action_type
	, count(*) over (partition by b.visit_id rows between unbounded preceding and unbounded following) as visit_cnt	
	, first_value(a.hits_page_path) over (partition by b.visit_id order by hits_seq) as first_page_path
from hits a
	join session b on a.visit_id = b.visit_id 
where visit_start_time >= (to_date('20170101', 'yyyymmdd') - interval '30 days') and visit_start_time < to_date('20170101', 'yyyymmdd')
and a.hits_type = 'PAGE'
), 
temp_02 as (
select hits_page_path
	, count(*) as page_cnt	
	, sum(case when visit_cnt = 1 then 1 else 0 end) as bounce_cnt_per_page	
	, count(distinct case when first_page_path = hits_page_path then visit_id else null end) as unique_visit_cnt_per_page	
from temp_01
group by hits_page_path
)
select *	
	, coalesce(round(100.0 * bounce_cnt_per_page / (case when unique_visit_cnt_per_page = 0 then null else unique_visit_cnt_per_page end), 2), 0) as bounce_pct	
from temp_02
order by page_cnt desc;
"""

df_bounce = pd.read_sql_query(sql=query, con=postgres_engine)
df_bounce2 = df_bounce.head(10)
df_bounce2

,hits_page_path,page_cnt,bounce_cnt_per_page,unique_visit_cnt_per_page,bounce_pct
0,/home,87960,21082,48591,43.39
1,/basket.html,24859,489,1196,40.89
2,/google+redesign/shop+by+brand/youtube,13357,3218,6537,49.23
3,/signin.html,11679,1194,3173,37.63
4,/google+redesign/apparel/men++s/men++s+t+shirts,8759,1099,2547,43.15
5,/store.html,7812,214,391,54.73
6,/google+redesign/apparel/men++s/men++s+outerwear,6795,480,1133,42.37
7,/asearch.html,6511,222,331,67.07
8,/google+redesign/apparel,5612,273,541,50.46
9,/google+redesign/electronics,4417,277,489,56.65


In [110]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='page_counts', x=df_bounce2['hits_page_path'], y=df_bounce2['page_cnt']), secondary_y=False)
fig.add_trace(go.Scatter(name='page_bounce_rate', x=df_bounce2['hits_page_path'], y=df_bounce2['bounce_pct']), secondary_y=True)

In [156]:
# past 30 days bounce rate by channel

query = """
with 
temp_01 as ( 
select hits_page_path, b.visit_id, b.channel_group, hits_seq, hits_type, hits_action_type	
	, count(*) over (partition by b.visit_id rows between unbounded preceding and unbounded following) as user_cnt	
	, first_value(hits_page_path) over (partition by b.visit_id order by hits_seq) as first_page_path
from hits a
	join session b on a.visit_id = b.visit_id 
where visit_start_time >= (to_date('20170101', 'yyyymmdd') - interval '30 days') and visit_start_time < to_date('20170101', 'yyyymmdd')
and a.hits_type = 'PAGE'
)
select channel_group, count(*) as page_cnt	 
	, sum(case when user_cnt = 1 then 1 else 0 end) as bounce_user_cnt
	, count(distinct visit_id) as unique_user_cnt
	, round(100.0 * sum(case when user_cnt = 1 then 1 else 0 end) / count(distinct visit_id), 2) as bounce_pct
from temp_01
group by channel_group
order by page_cnt desc;
"""

df_channel_bounce = pd.read_sql_query(sql=query, con=postgres_engine)
df_channel_bounce.head()

,channel_group,page_cnt,bounce_user_cnt,unique_user_cnt,bounce_pct
0,Organic Search,138592,13863,30652,45.23
1,Referral,80809,2021,10447,19.35
2,Direct,58471,6442,13711,46.98
3,Social,31637,8511,15170,56.10
4,Paid Search,19373,1300,3534,36.79


In [157]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='page count', x=df_channel_bounce['channel_group'], y=df_channel_bounce['page_cnt']), secondary_y=False)
fig.add_trace(go.Scatter(name='channel_bounce_rate', x=df_channel_bounce['channel_group'], y=df_channel_bounce['bounce_pct']), secondary_y=True)

In [145]:
# past 30 days bounce rate of channel per page 

query = """
with 
temp_01 as ( 
select hits_page_path, b.visit_id, b.channel_group, hits_seq, hits_type, hits_action_type	
	, count(*) over (partition by b.visit_id rows between unbounded preceding and unbounded following) as user_cnt	
	, first_value(hits_page_path) over (partition by b.visit_id order by hits_seq) as first_page_path
from hits a
	join session b on a.visit_id = b.visit_id 
where visit_start_time >= (to_date('20170101', 'yyyymmdd') - interval '30 days') and visit_start_time < to_date('20170101', 'yyyymmdd')
and a.hits_type = 'PAGE'
), 
temp_02 as (
select channel_group, hits_page_path, count(*) as page_cnt	
	, sum(case when user_cnt = 1 then 1 else 0 end) as bounce_cnt_per_channel	
	, count(distinct case when first_page_path = hits_page_path then visit_id else null end) as user_cnt_per_channel
	from temp_01
group by channel_group, hits_page_path
)
select *	
	, coalesce(round(100.0 * bounce_cnt_per_channel / (case when user_cnt_per_channel = 0 then null else user_cnt_per_channel end), 2), 0) as bounce_pct
from temp_02
order by page_cnt desc, hits_page_path, channel_group;
"""

df_channel_bounce = pd.read_sql_query(sql=query, con=postgres_engine)
df_channel_bounce2 = df_channel_bounce.head(50)
df_channel_bounce2.head()

,channel_group,hits_page_path,page_cnt,bounce_cnt_per_channel,user_cnt_per_channel,bounce_pct
0,Organic Search,/home,36232,7437,17135,43.40
1,Social,/home,20301,7860,14011,56.10
2,Direct,/home,11534,3136,7016,44.70
3,Referral,/home,11502,1006,6606,15.23
4,Organic Search,/google+redesign/shop+by+brand/youtube,8749,2477,5320,46.56


In [148]:
df_channel_bounce2['hits_simple_path'] = df_channel_bounce2['hits_page_path'].apply(lambda x: x.rsplit('/', 1)[-1] if '/' in x else x)
df_channel_bounce2.loc[:, 'hits_page_path'] = df_channel_bounce2['hits_simple_path']
df_channel_bounce2.drop(columns=['hits_simple_path'], inplace=True)
df_channel_bounce2.head()

C:\Users\DS\AppData\Local\Temp\ipykernel_1868\741025791.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\DS\AppData\Local\Temp\ipykernel_1868\741025791.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,channel_group,hits_page_path,page_cnt,bounce_cnt_per_channel,user_cnt_per_channel,bounce_pct
0,Organic Search,home,36232,7437,17135,43.40
1,Social,home,20301,7860,14011,56.10
2,Direct,home,11534,3136,7016,44.70
3,Referral,home,11502,1006,6606,15.23
4,Organic Search,youtube,8749,2477,5320,46.56


In [150]:
import plotly.express as px

fig = px.scatter(
    df_channel_bounce2,
    x='hits_page_path',
    y='bounce_pct',
    color='channel_group',
    title='Bounce Percentage by Page Path',
    labels={'hits_page_path': 'Page Path', 'bounce_pct': 'Bounce Percentage'},
    hover_data=['user_cnt_per_channel']
)

fig.show()